In [1]:
from hwt.hdl.constants import Time
from hwt.simulator.simTestCase import SimTestCase
from hwt.synthesizer.unit import Unit
from hwt.interfaces.std import Signal


# example component from previous tutorial
class DirectWire(Unit):
    def _declr(self):
        self.a = Signal()
        self.b = Signal()._m()

    def _impl(self):
        self.b(self.a)

        
# SimTestCase is derived from unittest.TestCase
# which is class of unit test framework from standard python library
class WireTC(SimTestCase):

    # if method name starts with "test" unittest framework
    # know that this method is test
    def test_simple(self):
        # create a unit instance
        u = DirectWire()
        # convert it to rtl level
        # decorate interface with agents (._ag property) which will
        # drive or monitor values on the interface
        self.compileSimAndStart(u)

        # there we have our test data, because SimpleUnit has only connection inside
        # None represents invalid value (like universal "x" in vhdl)
        inputData = [0, 1, 0, 1, None, 0, None, 1, None, 0]

        # add inputData to agent for interface "a"
        # now agent of "a" will popleft data from input data
        # and it will put them on interface "a"
        u.a._ag.data.extend(inputData)

        # now we run simulation, we use our unit "u", our monitors
        # and drivers of interfaces stored in "procs",
        # we save dum of value changes into file "tmp/simple.vcd"
        # (which is default) and we let simulation run for 100 ns
        self.runSim(100*Time.ns)

        # now we use part of unittest framework to check results
        # use assertValSequenceEqual which sill automatically convert
        # value objects to integer representation and checks them
        self.assertValSequenceEqual(u.b._ag.data, inputData)

In [2]:
import unittest

# this is how you run unittest.TestCase (there is nothing specific to HWT SimTestCase)
suite = unittest.TestSuite()

# this is how you can select specific test
# suite.addTest(WireTC('test_simple'))

# this is how you add all test from testcase
suite.addTest(unittest.makeSuite(WireTC))
runner = unittest.TextTestRunner(verbosity=3)
runner.run(suite)

test_simple (__main__.WireTC) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [3]:
# there is an vcd file generated in tmp directory by default
# (this behavior is specified by DEFAULT_LOG_DIR and DEFAULT_SIMULATOR property on test case)
!ls tmp

FifoTC_test_normalOp.json  WireTC_test_simple.json
FifoTC_test_normalOp.vcd   WireTC_test_simple.vcd


In [4]:
# Another way how to view and simulation output is to use HwtSignalDumpWidget
from jupyter_widget_hwt import HwtSignalDumpWidget

selected_test = WireTC('test_simple')
trace = HwtSignalDumpWidget(selected_test, width=1000, height=150)
display(trace)

HwtSignalDumpWidget(height='150px', value={'name': 'WireTC_test_simple__DirectWire', 'type': {'name': 'struct'…